# TV Script Generation
Neste projeto, você gerará seus próprios [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) Scripts de TV usando RNNs. Você estará usando parte do [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) ode scripts de 27 temporadas. A Rede Neural que você irá construir irá gerar um novo roteiro de TV para uma cena na [Taverna do Moe](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
Os dados já são fornecidos para você. Você estará usando um subconjunto do conjunto de dados original. Consiste apenas nas cenas da Taverna de Moe. Isso não inclui outras versões da taverna, como "Moe's Cavern", "Flame Moe's", "Uncle Moe's Family Feed-Bag", etc.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Exploração dos dados
Brinque com `view_sentence_range` para ver diferentes partes dos dados.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implementar funções de pré-processamento
A primeira coisa a fazer em qualquer conjunto de dados é o pré-processamento. Implemente as seguintes funções de pré-processamento abaixo:
- Tabela de pesquisa
- Tokenize Punctuation

### Tabela de pesquisa
Para criar uma incorporação de palavras, primeiro você precisa transformar as palavras em ids. Nesta função, crie dois dicionários:
- Dicionário para ir das palavras para um id, vamos chamar `vocab_to_int`
- Dicionário para ir do id para a palavra, vamos chamar `int_to_vocab`

Devolve estes dicionários na seguinte tupla `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    # Use comprenhension lists to build our dictionaries.
    vocab_to_int = {word:idx for idx, word in enumerate(vocab)}
    int_to_vocab = {idx:word for idx, word in enumerate(vocab)}
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
Nós dividiremos o script em uma matriz de palavras usando espaços como delimitadores. No entanto, pontuações como períodos e pontos de exclamação tornam difícil para a rede neural distinguir entre a palavra "bye" e "bye!".

Implemente a função `token_lookup` para retornar um comando que será usado para simbolizar símbolos como"! " em "|| Exclamation_Mark ||". Crie um dicionário para os seguintes símbolos, onde o símbolo é a chave e o valor é o token:
- Período (.)
- vírgula (,)
- Aspa ( " )
- ponto e vírgula (;)
- Ponto de exclamação ( ! )
- Ponto de interrogação ( ? )
- Parênteses esquerdos (()
- Parênteses direitos ())
- traço (-)
- retorno (\ n)

Este dicionário será usado para simbolizar os símbolos e adicionar o delimitador (espaço) em torno dele. Isso separa os símbolos como sua própria palavra, facilitando a previsão da rede neural na próxima palavra. Certifique-se de não usar um token que possa ser confundido como uma palavra. Em vez de usar o token "dash", tente usar algo como "|| dash ||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {
        '.':  '<period>',
        ',':  '<comma>',
        '"':  '<quotation_mark>',
        ';':  '<semicolon>',
        '!':  '<exclamation_mark>',
        '?':  '<period>',
        '(':  '<left_parenthesis>',
        ')':  '<right_parenthesis>',
        '--': '<double_hyphen>',
        '\n': '<newline>'
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pré-processe todos os dados e salve-os
Executar a célula de código abaixo irá pré-processar todos os dados e salvá-los em um arquivo

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
Este é seu primeiro checkpoint. Se você decidir voltar a este notebook ou precisar reiniciar o notebook, poderá começar a partir daqui. Os dados pré-processados foram salvos no disco.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Construa a Rede Neural
Você construirá os componentes necessários para construir um RNN implementando as seguintes funções abaixo:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Verifique a versão do TensorFlow e acesse o GPU

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) <= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/home/marcus/miniconda3/envs/dog-project/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implemente a função `get_inputs ()` para criar Placeholders do TF para a Rede Neural. Deve criar os seguintes marcadores de posição:
- Espaço reservado para texto de entrada chamado "entrada" usando o [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) parâmetro `name`.
- Espaço reservado para alvos
- Espaço reservado para Learning Rate

Retorna os placeholders na seguinte tupla `(Input, Targets, LearningRate)`

In [17]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_ = tf.placeholder(tf.int32, shape=[None, None], name='input')
    targets_ = tf.placeholder(tf.int32, shape=[None, None], name='targets')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    return (input_, targets_, learning_rate_)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Construa uma célula RNN e inicialize
Empilhe um ou mais [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- O tamanho do Rnn deve ser definido usando `rnn_size`
- Inicialize o estado da célula usando o MultiRNNCell [`zero_state ()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna a célula e o estado inicial na seguinte tupla `(Cell, InitialState)`

In [18]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    num_layers = 2
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Incorporação de Palavras
Aplique a incorporação ao `input_data` usando o TensorFlow. Devolva a sequência incorporada.

In [19]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
YVocê criou uma célula RNN na função `get_init_cell ()`. Hora de usar a célula para criar um RNN.
- Construa o RNN usando o [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
- Aplique o nome "final_state" ao estado final usando [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna as saídas e o estado final_state na seguinte tupla `(Saídas, FinalState)`

In [20]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return (outputs, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Construa a Rede Neural
Aplique as funções implementadas acima para:
- Aplicar incorporação ao `input_data` usando sua função` get_embed (input_data, vocab_size, embed_dim) `.
- Construa RNN usando `cell` e sua função` build_rnn (cell, inputs) `.
- Aplique uma camada totalmente conectada com uma ativação linear e `vocab_size` como o número de saídas.

Retorna os logits e o estado final na seguinte tupla (Logits, FinalState)

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embedding_layer = get_embed(input_data, vocab_size, embed_dim)
    (outputs, final_state) = build_rnn(cell, embedding_layer)
    logits = tf.layers.dense(outputs, vocab_size, activation= None, use_bias=True)
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Lotes
Implemente `get_batches` para criar lotes de entrada e destinos usando` int_text`. Os lotes devem ser um array Numpy com a forma `(número de lotes, 2, tamanho do lote, comprimento da seqüência)`. Cada lote contém dois elementos:
- O primeiro elemento é um único lote de ** entrada ** com a forma `[tamanho do lote, comprimento da sequência]`
- O segundo elemento é um único lote de ** destinos ** com a forma `[tamanho do lote, comprimento da sequência]`

Se você não puder preencher o último lote com dados suficientes, elimine o último lote.

Por exemplo, `get_batches ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) `retornaria uma matriz Numpy dos seguintes:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Observe que o último valor de destino no último lote é o primeiro valor de entrada do primeiro lote. Neste caso, `1`. Essa é uma técnica comum usada na criação de lotes de sequências, embora não seja muito intuitiva:

In [13]:
import math
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    # Calculate the number of batches
    num_batches = len(int_text) // (batch_size  * seq_length)
    # Drop long batches. Transform into a numpy array and reshape it for our purposes
    np_text = np.array(int_text[:num_batches * (batch_size  * seq_length)])
    # Reshape the data to give us the inputs sequence.
    in_text = np_text.reshape(-1, seq_length)
    # Roll (shift) and reshape to get target sequences (maybe not optimal)
    tar_text = np.roll(np_text, -1).reshape(-1, seq_length)
    output = np.zeros(shape=(num_batches, 2, batch_size, seq_length), dtype=np.int)
    # Prepare the output
    for idx in range(0, in_text.shape[0]):
        jj = idx % num_batches
        ii = idx // num_batches
        output[jj,0,ii,:] = in_text[idx,:]
        output[jj,1,ii,:] = tar_text[idx,:]
    return output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Treinamento em Redes Neurais
### Hyperparameters
Ajuste os seguintes parâmetros:

- Defina `num_epochs` como o número de épocas.
- Configure `batch_size` para o tamanho do lote.
- Defina `rnn_size` para o tamanho dos RNNs.
- Configure `embed_dim` para o tamanho da incorporação.
- Defina `seq_length` como o comprimento da sequência.
- Configure `learning_rate` para a taxa de aprendizado.
- Defina `show_every_n_batches` com o número de lotes que a rede neural deve imprimir.

In [16]:
# Number of Epochs
num_epochs = 35
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Construir o gráfico
Construa o gráfico usando a rede neural que você implementou.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

NameError: name 'embed_dim' is not defined

## Train
Treine a rede neural nos dados pré-processados. Se você tiver dificuldade em obter uma boa perda, verifique os [fóruns] (https://discussions.udacity.com/) para ver se alguém está tendo o mesmo problema.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/44   train_loss = 8.821
Epoch   0 Batch   10/44   train_loss = 6.795
Epoch   0 Batch   20/44   train_loss = 6.163
Epoch   0 Batch   30/44   train_loss = 6.088
Epoch   0 Batch   40/44   train_loss = 5.870
Epoch   1 Batch    6/44   train_loss = 5.505
Epoch   1 Batch   16/44   train_loss = 5.426
Epoch   1 Batch   26/44   train_loss = 5.216
Epoch   1 Batch   36/44   train_loss = 5.207
Epoch   2 Batch    2/44   train_loss = 4.847
Epoch   2 Batch   12/44   train_loss = 4.925
Epoch   2 Batch   22/44   train_loss = 4.940
Epoch   2 Batch   32/44   train_loss = 4.822
Epoch   2 Batch   42/44   train_loss = 4.829
Epoch   3 Batch    8/44   train_loss = 4.707
Epoch   3 Batch   18/44   train_loss = 4.722
Epoch   3 Batch   28/44   train_loss = 4.717
Epoch   3 Batch   38/44   train_loss = 4.589
Epoch   4 Batch    4/44   train_loss = 4.535
Epoch   4 Batch   14/44   train_loss = 4.700
Epoch   4 Batch   24/44   train_loss = 4.503
Epoch   4 Batch   34/44   train_loss = 4.480
Epoch   5 

## Save Parameters
Salve `seq_length` e` save_dir` para gerar um novo script de TV.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implementar Gerar Funções
### Obtenha Tensores
Obtenha tensores de `loaded_graph` usando a função [` get_tensor_by_name () `] (https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name). Obtenha os tensores usando os seguintes nomes:
- "entrada: 0"
- "initial_state: 0"
- "final_state: 0"
- "probs: 0"

Retorna os tensores na seguinte tupla `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)`

In [25]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Escolher palavra
Implemente a função `pick_word ()` para selecionar a próxima palavra usando `probabilities`.

In [27]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    indices      = np.argsort(-probabilities)
    choice_probs = np.array([probabilities[idx] for idx in indices])
    renormaized_probs = choice_probs / choice_probs.sum()

    choice = np.random.choice(indices, p=renormaized_probs)

    return int_to_vocab[choice]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Gerar script de TV
Isso irá gerar o script de TV para você. Defina `gen_length` com o comprimento do script de TV que você deseja gerar.

In [28]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(annoyed) yeah, we can't let me clap her erasers.


lenny_leonard: yeah, you guys are eurotrash, i'd like a spy.
kemi: i didn't see nothin'.
homer_simpson: moe. sorry we are all the store to get applesauce for maggie two hours of all. how did i had too much.
laney_fontaine: so sweet.
moe_szyslak: uh, if your wife was tough.
bart_simpson: no. you're still talking to feel you on the house, right. it was before... going late.
homer_simpson: come off, homer. hottest he brought a carl's company rascals.
moe_szyslak: now, how do i have you a person it.
homer_simpson: what's that.
barney_gumble: i'm gonna catch you...
homer_simpson: hey, moe! you called me to be my cyrano sack and they admit it.
homer_simpson:(rolls dog) you're too late, so i can liven it up.
moe_szyslak: how much you get over now.
harv: we hope she works.
moe_szyslak: really


# O roteiro de TV é absurdo
Tudo bem se o roteiro da TV não fizer sentido. Nós treinamos com menos de um megabyte de texto. Para obter bons resultados, você terá que usar um vocabulário menor ou obter mais dados. Felizmente há mais dados! Como mencionamos no início deste projeto, este é um subconjunto de [outro conjunto de dados] (https://www.kaggle.com/wcukierski/the-simpsons-by-the-data). Nós não fizemos você treinar em todos os dados, porque isso levaria muito tempo. No entanto, você está livre para treinar sua rede neural em todos os dados. Depois de concluir o projeto, é claro.
# Enviando este projeto
Ao enviar este projeto, certifique-se de executar todas as células antes de salvar o bloco de anotações. Salve o arquivo do notebook como "dlnd_tv_script_generation.ipynb" e salve-o como um arquivo HTML em "File" -> "Download as". Inclua os arquivos "helper.py" e "problem_unittests.py" no seu envio.